# Teoria

### Do czego to potrzebne?

Wektorów i wartości własnych możemy użyć na przykład do redukcji wymiarowości, na ich podstawie oceniając istotność zmiennych

### Przekształcenia liniowe (linear transformation)

Endomorfizmem liniowym (rodzaj przekształcenia liniowego) jest funkcja $f(\textbf{w})$ operująca na wektorze kolumnowym długości $n$ postaci$$f(\textbf{w})=A\textbf{w},$$ gdzie $A$ jest macierzą kwadratową rozmiaru $n\times n$.

Dla $n=2$ endomorfizm liniowy jest zdefiniowany jako

![](../img/2023-03-26-20-14-09.png)

Jeżeli każdy punkt na obrazie potraktujemy jako wektor $[x,y]$, możemy zobrazować jak działa przykładowy endomorfizm liniowy.


![](../img/2023-03-26-19-17-31.png)

![](../img/2023-03-26-19-21-14.png)

![](../img/2023-03-26-19-21-45.png)

### Wektory własne

Wektor własny wskazuje kierunek skalowania/rotacji jaki wykonuje przekształcenie liniowe
$$f(\textbf{w})=A\textbf{w},$$
Gdzie wzor powyżej opisuje konkretne przekształcenie (przemnożenie wektora W przez macierz przekształceń A)

![](../img/2023-03-26-19-24-22.png)

Natomiast **wartością własną** i związanym z nią **wektorem własnym** nazywamy wartość $\lambda$ i wektor $\mathbf{w}_0$ spełniające warunek$$f(\mathbf{w}_0)=\lambda \mathbf{w_0}$$Gdzie w0 opisuje wektor który określa kierunek w jakim zwrócony był wektor z któego wyznaczono wektor własny.
```
Wektor, który po przeskalowaniu (przemnożenie prez WARTOŚĆ WŁASNĄ) wskazuje ten sam kierunek co oryginalny wektor, jest jego wektorem własnym.

### W pythonie

In [1]:
import numpy as np
A = np.array([[ 1, 2, 3], [ 4, 5, 6], [ 7, 8, 22]])

wartosci_wlasne, wektory_wlasne = np.linalg.eig(A)
wartosci_wlasne, wektory_wlasne

(array([25.59639374, -0.52098034,  2.9245866 ]),
 array([[-0.13957085, -0.84384918, -0.21345016],
        [-0.30183948,  0.53152606, -0.87052838],
        [-0.9430869 ,  0.0734753 ,  0.44341782]]))

### Więcej informacji:
- https://www.kowalskimateusz.pl/wektory-i-wartosci-wlasne-eigenvalues-and-eigenvectors/ (Stąd m.in zaczerpnięto grafikę wykresu)
- https://www.youtube.com/watch?v=qUeud6DvOWI (Artykul powyzej w wersji video)

# Przykładowe użycie w DataScience (PCA)

### Wstęp do PCA

Wykorzystanie do redukcji wymiarowości zbioru danych (Używane w algorytmie PCA - Princial component analysis = Analiza głównych składowych):

#### 0. Co robi PCA (bardzo ogólnie i skrótowo)

Bierze duża liczbę zmiennych (np. 50) i kompresuje je do małej (2) z zachowaniem możliwie jak największej wartości opisującej 

=============================================================================================================

KIEDYŚ PCA było używane do kompresji danych i przyśpieszania obliczeń, ale to traci na popularności. Cytat Andrew Ng: 

"But it turns out with modern machine learning algorithms, algorithms like deep learning, this doesn't actually hold that much,
and is much more common to just take the high-dimensional dataset, and feed it into say your neural network rather than run PCA because PCA has some computational cost as well.

You may hear about this in some of the other research papers, but I don't really see this done much anymore.
But the most common thing that I use PCA for today is visualization and then I find it very useful to reduce the dimensional data to visualize it."

=============================================================================================================

(Grafika z CURSERY)

![](../img/2023-03-27-00-36-44.png)

- na Z1 mogą się składać np. featury opisujące wielkość kraju (Liczba kilometrów, liczba miast, liczba ludzi, ...)
- na Z2 mogą się składać np. featury opisujące jakość życia każdego obywatela (przychód na obywatela, ilosc mieszkań na człowieka, ...)

Ogólnie grupowane są podobne zmienne i to pozwala wizualizować wielowymiarowe dane w sposób interpretowalny przez człowieka

Potem na podstawie macierzy wariancji wyznaczana jest taka oś Z (principal component) która będzie miała maksymalną wariancję 

 (Xy rzutowane na tą oś będą jak najmniej na siebie nachodzić, odstępy między nimi będą największe)
 
![](../img/2023-03-27-00-41-34.png)

Do konwersji feature'ów należy:
- Znaleźc linię która najlepiej rozdziela Xy
- Znajdź wektor o długości 1 który leży na tej linii
- Wylicz dot-product starych fature'ów oraz tego wektora

Kolejne linie (po wyznaczeniu pierwszej wg wariancji) dobieramy pod kądem 90 stopni do poprzednich

### Przykład zastosowania macierzy kowariancji / wektorów własncyh / wartości własncyh

#### 1. Wczytanie jakichkolwiek danych

In [2]:
import pandas as pd
csv_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['Sepal_Length','Sepal_Width','Petal_Length','Petal_Width','Class']
df =  pd.read_csv(csv_url, names = col_names)
features = df[['Sepal_Length', 'Sepal_Width','Petal_Length','Petal_Width']]
targets = df['Class']
features

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


#### 2. Wyznaczamy macierz kowariancji (czyli zależność pomiędzy poszczególnymi kolumnami):

In [3]:
macierz_kowariancji = np.cov(features.T)
macierz_kowariancji

array([[ 0.68569351, -0.03926846,  1.27368233,  0.5169038 ],
       [-0.03926846,  0.18800403, -0.32171275, -0.11798121],
       [ 1.27368233, -0.32171275,  3.11317942,  1.29638747],
       [ 0.5169038 , -0.11798121,  1.29638747,  0.58241432]])

Kowariancja mierzy, jak bardzo dwie zmienne są ze sobą powiązane i jak bardzo jedna zmienna wpływa na wartość drugiej zmiennej.
Macierz kowariancji zawiera informacje o kowariancjach pomiędzy wszystkimi parami zmiennych w zbiorze danych.

Kowariancja, cov⁡(X,Y) – liczba określająca odchylenie elementów od sytuacji idealnej, w której występuje zależność liniowa. Zależność tę określa się między zmiennymi losowymi X i Y.

![](../img/2023-03-27-09-49-33.png)

W macierzy kowariancji:
- elementy po przekątnej odpowiadają wariancji każdej zmiennej, czyli mierzą jak bardzo wartości zmieniają się wokół swojego średniego poziomu. 
- Elementy poza przekątną to kowariancje pomiędzy parami zmiennych, czyli mierzą w jaki sposób zmiany jednej zmiennej są skorelowane ze zmianami drugiej zmiennej.

#### 3. Wyznaczamy wartości i wektory własne macierzy kowariancji:

In [4]:
wartosci_wlasne, wektory_wlasne = np.linalg.eig(macierz_kowariancji)
wartosci_wlasne, wektory_wlasne

(array([4.22484077, 0.24224357, 0.07852391, 0.02368303]),
 array([[ 0.36158968, -0.65653988, -0.58099728,  0.31725455],
        [-0.08226889, -0.72971237,  0.59641809, -0.32409435],
        [ 0.85657211,  0.1757674 ,  0.07252408, -0.47971899],
        [ 0.35884393,  0.07470647,  0.54906091,  0.75112056]]))

#### 4. Sortujemy wartości własne malejąco. 
Te, które mają większe wartości, odpowiadają kierunkom w przestrzeni, w których zmienność danych jest większa (a więc zawierają najwięcej informacji)

In [5]:
indeksy_sort = wartosci_wlasne.argsort()[::-1] #argsort zwraca numery indexow od najmniejszej wartosci, [::-1] odwraca kolejnosc
wartosci_wlasne_posortowane = wartosci_wlasne[indeksy_sort]
wektory_wlasne_posortowane = wektory_wlasne[:, indeksy_sort]
wartosci_wlasne_posortowane, wektory_wlasne_posortowane

(array([4.22484077, 0.24224357, 0.07852391, 0.02368303]),
 array([[ 0.36158968, -0.65653988, -0.58099728,  0.31725455],
        [-0.08226889, -0.72971237,  0.59641809, -0.32409435],
        [ 0.85657211,  0.1757674 ,  0.07252408, -0.47971899],
        [ 0.35884393,  0.07470647,  0.54906091,  0.75112056]]))

#### 5. Wykorzystujemy wektory własne do redukcji wymiarowości. 
- Możemy wybrać kilka największych wartości własnych i odpowiadające im wektory własne, a następnie pomnożyć macierz danych przez te wektory własne. 
- Otrzymamy w ten sposób nową macierz danych z mniejszą ilością wymiarów

In [6]:
najwieksze_wartosci_wlasne = wartosci_wlasne_posortowane[:2]
najwieksze_wektory_wlasne = wektory_wlasne_posortowane[:, :2]
new_features = features.dot(najwieksze_wektory_wlasne)
new_features

,0,1
0,2.827136,-5.641331
1,2.795952,-5.145167
2,2.621524,-5.177378
3,2.764906,-5.003599
4,2.782750,-5.648648
...,...,...
145,7.455360,-5.502139
146,7.037007,-4.939703
147,7.275389,-5.393243
148,7.412972,-5.430600


W ten sposób uzyskujemy nową macierz danych, która jest mniejsza wymiarowo, a więc łatwiejsza do przetworzenia.

#### 6. Przykład szybkiej nauki regresjii logistycznej dla zewryfikowania, czy dane po redukcji wymiarowości dalej mają wartość opisującą

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Ten warning informuje, że dobrze byłoby normalizować dane dla osiągnięcia lepszych wyników. Na potrzeby prezentacji przykładu możemy to zignorować
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter
simplefilter("ignore", category=ConvergenceWarning)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(new_features, targets, test_size=0.2, random_state=101)

# Trenowanie modelu
model = LogisticRegression()
model.fit(X_train, y_train)

# Ocena dokładności modelu
y_pred = model.predict(X_test)
for id, row in enumerate(zip(y_test, y_pred)):
    print('target vs prediction: ', row)
    if id>10: break

score = model.score(X_test, y_test)
print("\nDokładność modelu po redukcji wymiarów:", score)

# # DLA PORÓWNANIA MODEL WYTRENOWANY NA ORYGINALNYCH DANYCH (BEZ REDUKCJI WYMIARÓW):
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=101)
model = LogisticRegression()
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print("Dokładność modelu z oryginalnych danych:", score)

print("\nUwaga - wyniki modeli sa uzaleznione od random-state. \nDla niektórych wynik po redukcji będzie jednakowy albo nawet lepszy niż przed!")

target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-versicolor', 'Iris-versicolor')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')
target vs prediction:  ('Iris-virginica', 'Iris-virginica')
target vs prediction:  ('Iris-setosa', 'Iris-setosa')

Dokładność modelu po redukcji wymiarów: 0.9666666666666667
Dokładność modelu z oryginalnych danych: 1.0

Uwaga - wyniki modeli sa uzaleznione od random-state. 
Dla niektórych wynik po redukcji będzie jednakowy albo nawet lepszy niż przed!


# PCA z pakietem sklearn

In [43]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(features)
# Wypisanie macierzy kowariancji, wektorów własncyh i wartości własncyh
print(f"{pca.get_covariance()}\n{pca.components_}\n{pca.explained_variance_}")

print('\nwartośc objaśniająca dla każdej z kolumn:', pca.explained_variance_ratio_)
print('Jak widać 1 kolumna ma 92% wartości objaśniające, druga 5% \nRazem mają ~97% tak odrzucajac 2 kolumny (połowę) tracimy tylko 3% informacji')

new_features_pca = pca.transform(features)
print("\nFragment danych po redukcji wymiarów:\n", new_features_pca[:10])
# Tranfsormacje można odwrócić z użyciem pca.inverse_transform(new_features_pca), ale część informacji (~3% w tym wypadku) jest tracona
# pca.inverse_transform(new_features_pca)

[[ 0.67919741 -0.03258618  1.27066452  0.5321852 ]
 [-0.03258618  0.18113034 -0.31863564 -0.13363564]
 [ 1.27066452 -0.31863564  3.11934547  1.28541527]
 [ 0.5321852  -0.13363564  1.28541527  0.58961806]]
[[ 0.36158968 -0.08226889  0.85657211  0.35884393]
 [ 0.65653988  0.72971237 -0.1757674  -0.07470647]]
[4.22484077 0.24224357]

wartośc objaśniająca dla każdej z kolumn: [0.92461621 0.05301557]
Jak widać 1 kolumna ma 92% wartości objaśniające, druga 5% 
Razem mają ~97% tak odrzucajac 2 kolumny (połowę) tracimy tylko 3% informacji

Fragment danych po redukcji wymiarów:
 [[-2.68420713  0.32660731]
 [-2.71539062 -0.16955685]
 [-2.88981954 -0.13734561]
 [-2.7464372  -0.31112432]
 [-2.72859298  0.33392456]
 [-2.27989736  0.74778271]
 [-2.82089068 -0.08210451]
 [-2.62648199  0.17040535]
 [-2.88795857 -0.57079803]
 [-2.67384469 -0.1066917 ]]


In [46]:
# porównanie wyznaczonych wartości z poprzednio wygenerowanymi
print("Macierz kowariancji:")
print('Wyznaczona przez: np.cov(features.T)\n', macierz_kowariancji)
print('Wyciągnięte z PCA przez: pca.get_covariance()\n', pca.get_covariance())

print("\nWektory własne:")
print('Wyznaczona przez: np.linalg.eig(macierz_kowariancji)\n', wektory_wlasne)
print('Wyciągnięte z PCA przez: pca.components_\n', pca.components_)

Macierz kowariancji:
Wyznaczona przez: np.cov(features.T)
 [[ 0.68569351 -0.03926846  1.27368233  0.5169038 ]
 [-0.03926846  0.18800403 -0.32171275 -0.11798121]
 [ 1.27368233 -0.32171275  3.11317942  1.29638747]
 [ 0.5169038  -0.11798121  1.29638747  0.58241432]]
Wyciągnięte z PCA przez: pca.get_covariance()
 [[ 0.67919741 -0.03258618  1.27066452  0.5321852 ]
 [-0.03258618  0.18113034 -0.31863564 -0.13363564]
 [ 1.27066452 -0.31863564  3.11934547  1.28541527]
 [ 0.5321852  -0.13363564  1.28541527  0.58961806]]

Wektory własne:
Wyznaczona przez: np.linalg.eig(macierz_kowariancji)
 [[ 0.36158968 -0.65653988 -0.58099728  0.31725455]
 [-0.08226889 -0.72971237  0.59641809 -0.32409435]
 [ 0.85657211  0.1757674   0.07252408 -0.47971899]
 [ 0.35884393  0.07470647  0.54906091  0.75112056]]
Wyciągnięte z PCA przez: pca.components_
 [[ 0.36158968 -0.08226889  0.85657211  0.35884393]
 [ 0.65653988  0.72971237 -0.1757674  -0.07470647]]


In [47]:
print("\nWartości własne:")
print('Wyznaczona przez: np.linalg.eig(macierz_kowariancji)\n', wartosci_wlasne)
print('Wyciągnięte z PCA przez: pca.explained_variance_\n', pca.explained_variance_)

print("\nDane po redukcji wymiarów:")
print('Wyznaczona przez: features.dot(najwieksze_wektory_wlasne\n', new_features[:5])
print('Wyznaczona przez PCA: pca.transform(features)\n', new_features_pca[:5])


Wartości własne:
Wyznaczona przez: np.linalg.eig(macierz_kowariancji)
 [4.22484077 0.24224357 0.07852391 0.02368303]
Wyciągnięte z PCA przez: pca.explained_variance_
 [4.22484077 0.24224357]

Dane po redukcji wymiarów:
Wyznaczona przez: features.dot(najwieksze_wektory_wlasne
           0         1
0  2.827136 -5.641331
1  2.795952 -5.145167
2  2.621524 -5.177378
3  2.764906 -5.003599
4  2.782750 -5.648648
Wyznaczona przez PCA: pca.transform(features)
 [[-2.68420713  0.32660731]
 [-2.71539062 -0.16955685]
 [-2.88981954 -0.13734561]
 [-2.7464372  -0.31112432]
 [-2.72859298  0.33392456]]
